# Quantum Signal Processing and Quantum Singular Value Transformation
*Copyright (c) 2022 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Overview

**Quantum signal processing** (QSP), originally purposed by Guang Hao Low and Issac L. Chuang [[1]](https://quantum-journal.org/papers/q-2019-07-12-163/), is an algorithm for simulation of polynomial transformation of scalars using reflection ($R_X$) and rotation ($R_Z$) operators. QSP states that, for a specific group of polynomials like the Chebyshev polynomials of the first kind, such simulation can be done by a single qubit. Moreover, using a technique called "linear combination of unitaries", we can use two more ancilla qubit to simulate all complex polynomials and hence approximate any functions that can be expanded by Taylor series.

The idea of QSP is further generalized by paper [[2]](https://doi.org/10.1145/3313276.3316366), so that it can simulate polynomial transformation of matrices using high-dimensional rotation operators. Since the polynomial transformation of a diagonalizable matrix is intrinsically the polynomial transformation of its singular values, this algorithm is called the **quantum singular value transformation** (QSVT).

Moreover, a technique called **qubitization** is further employed to substantially reduce the complexity of high-dimensional rotation operators, so that one (ancilla) qubit suffices to perform the rotation in larger space.

Based on paper [[2]](https://doi.org/10.1145/3313276.3316366), this tutorial will illustrate quantum signal processing, quantum singular value transformation and qubitization, and how to implement these algorithms in PaddleQuantum. But first of all, we need to present the idea of block encoding.

Here are some necessary libraries and packages.

In [17]:
import numpy as np
from numpy.polynomial.polynomial import Polynomial
from numpy.polynomial import Chebyshev
import paddle

# PaddleQuantum related packages
import paddle_quantum
from paddle_quantum.ansatz import Circuit
from paddle_quantum.qinfo import dagger
from paddle_quantum.linalg import unitary_random_with_hermitian_block, density_matrix_random

from paddle_quantum.qsvt import poly_matrix, ScalarQSP, Phi_verification, reflection_based_quantum_signal_processing, QSVT

## Block Encoding

Block encoding is a method to encode matrices of data. In quantum computing, for a matrix $A \in \mathbb{C}^{2^n \times 2^n}$, a unitary $U \in \mathbb{C}^{2^m \times 2^m}$ is said to be a **block encoding** of $A$ if there exists two orthogonal projectors $W, V \in \mathbb{C}^{2^m \times 2^m}$ such that

$$
W U V = 
\begin{bmatrix}
    A & 0 \\
    0 & 0
\end{bmatrix}
= A \oplus 0I^{\otimes (m - n)}. \tag{1}
$$

This could appear more familiar when $W = V = |0\rangle^{\otimes (m - n)}\langle0|^{\otimes (m - n)} \otimes I^{\otimes n}$, where above equation implies

$$
U = \begin{bmatrix}
    A & ... \\
    ... & ...
\end{bmatrix}. \tag{2}
$$

That is, $A$ is the left-upper block of matrix $U$.

### Encoding and Decoding of Matrices

There are numerous ways to find a block encoding unitary $U$ of a matrix $A$. For example, when $A$ is diagonalizable, $U$ can be constructed as

$$
U = \begin{bmatrix}
    A & i\sqrt{I^{\otimes n} - A^2} \\
    i\sqrt{I^{\otimes n} - A^2} & A
\end{bmatrix}; \tag{3}
$$

when $A$ is a unitary, we can simply select $U = A \oplus I^{\otimes (m - n)}$ i.e. $U$ is a controlled $A$. Despite the fact that we do not hold a mature method for doing this in a quantum system, several studies [[3]](http://arxiv.org/abs/2203.10236) [[4]](http://arxiv.org/abs/2206.03505) have been conducted on the quantum implementation of block encodings. In this tutorial, we will assume that there exists an efficient algorithm to block encode a matrix (or at least a Hermitian matrix) in a quantum circuit.

#### Realization in PaddleQuantum.

In [18]:
num_qubits = 3
num_block_qubits = 2

# create a 3-qubit block encoding unitary U of a random 2-qubit Hermitian matrix A
U = unitary_random_with_hermitian_block(num_qubits)
A = U[0: 2 ** num_block_qubits, 0: 2 ** num_block_qubits]

As for the decoding part, the information of matrix $A \in \mathbb{C}^{2^n \times 2^n}$ can be extracted by applying its block encoding unitary $U \in \mathbb{C}^{2^m \times 2^m}$ on $|0\rangle^{\otimes (m - n)}\langle0|^{\otimes (m - n)} \otimes \rho$, where $\rho$ is an $n$-qubit quantum state. Then the output state is

$$
U (|0\rangle^{\otimes (m - n)}\langle0|^{\otimes (m - n)} \otimes \rho) U^\dagger
= \begin{bmatrix}
    A \rho A^\dagger & ... \\
    ... & ...
\end{bmatrix}
= |0\rangle^{\otimes (m - n)}\langle0|^{\otimes (m - n)} \otimes A \rho A^\dagger + (I^{\otimes (m - n)} - |0\rangle^{\otimes (m - n)}\langle0|^{\otimes (m - n)}) \otimes ... \tag{4}
$$

Measuring the first quantum register with outcome $0^{\otimes (m - n)}$ gives the desired information.

![block-decoding](figures/QSVT-fig-block-decoding.png "Figure 1: Graph Illustration of Block Decoding")

The success probability of decoding is the probability of measuring $0^{\otimes (m - n)}$, which can be exponentially small as $m - n$ increases. However, this would no longer be a problem if we can control the size of first register. In this tutorial we will assume $m = n + 1$.

In [19]:
# set density matrix backend
paddle_quantum.set_backend('density_matrix')

# define input state
rho = density_matrix_random(num_block_qubits)
zero_state = paddle.eye(2 ** (num_qubits - num_block_qubits), 1) # create a 0 state (in state_vector form)
input_state = paddle_quantum.State(paddle.kron(zero_state @ dagger(zero_state), rho))

# define auxiliary register
aux_register = list(range(num_qubits - num_block_qubits))

# construct the circuit
cir = Circuit(num_qubits)
cir.oracle(U, list(range(num_qubits)))
cir.collapse(aux_register, desired_result='0', if_print = True) # call Collapse operator

# get output_state and actual output rho
output_state = cir(input_state)
output_rho = output_state.data[0: 2 ** num_block_qubits, 0: 2 ** num_block_qubits]

qubits [0] collapse to the state |0> with probability 0.25072506070137024


We can verify that the ``output_rho`` is the same as $\frac{A \rho A^\dagger}{\text{Tr}(A \rho A^\dagger)}$.

In [20]:
expect_rho = A @ rho @ dagger(A)
expect_rho /= paddle.trace(expect_rho)
print(f"the difference between input and output is {paddle.norm(paddle.abs(expect_rho - output_rho)).item()}")

the difference between input and output is 0.0


When $m = 1$ (and hence $n = 0$), $A$ is a scalar. In this case we can calculate the expectance of $U$ in terms of state $|0\rangle$ to receive $A$.

## Quantum Signal Processing

Theorem 3-4 in paper [[2]]((https://doi.org/10.1145/3313276.3316366)) state that, for a polynomial $P \in \mathbb{C}[x]$ with $\deg(P) = k$ that satisfies

- if $k$ is even/odd, then $P$ is a even/odd polynomial;
- $|P(x)| \leq 1, \,\forall x \in [-1, 1]$;
- $|P(x)| \geq 1, \,\forall x \in (-\infty, -1] \cup [1, \infty)$;
- if $k$ is even, then $P(ix)P^*(ix) \geq 1$,

there exists a polynomial $Q \in \mathbb{C}[x]$ and a vector of angles $\Phi = (\phi_0, ..., \phi_k) \in \mathbb{R}^{k + 1}$ such that $\forall x \in [-1, 1]$,

$$
W_\Phi(x) := R_Z(-2\phi_0) \prod_{j = 1}^k R_X(\arccos(x)) R_Z(-2\phi_j)
= \begin{bmatrix}
   P(x) & i Q(x)\sqrt{1 - x^2} \\
   iQ^*(x)\sqrt{1 - x^2} & P^*(x) 
\end{bmatrix}. \tag{5}
$$

That is, we can use $k + 1$ rotation ($R_Z$) gates and $k$ reflection ($R_X$) gates to receive a block encoding unitary $W_\Phi(x)$ of $P(x)$. After block decoding we have completed the simulation of $P(x)$.

Here are some polynomials that satisfy above requirements. 

In [21]:
# simple odd polynomials
# P = Polynomial([0, 0, 0, 0, 0, 1])
# P = Polynomial([0, 0.5, 0, 0, 0, 0.5])
P = Polynomial([0, 1 / 3, 0, 0, 0, 1 / 3, 0, 1 / 3])

# Chebyshev polynomials of first kind with degree 10
# P = Chebyshev([0 for _ in range(10)] + [1]).convert(kind=Polynomial)

# Chebyshev polynomials of first kind with degree 11
# P = Chebyshev([0 for _ in range(11)] + [1]).convert(kind=Polynomial)

Note that such structure of $W_\Phi$ naturally fits the family of Chebyshev polynomials. Indeed, for a Chebyshev polynomials of first kind with order $k$, its corresponding $\Phi$ is $(0, \pi, ..., \pi)$ (if $k$ is even) or $(\pi, ..., \pi)$ (if $k$ is odd). 

PaddleQuantum has a built-in class `ScalarQSP` for quantum signal processing and a function `Phi_verification` to verify whether $W_\Phi$ is a block encoding of $P$.

In [22]:
qsp = ScalarQSP(P)
print(qsp.Phi)
assert Phi_verification(qsp.Phi.numpy(), P)

Tensor(shape=[8], dtype=float64, place=Place(cpu), stop_gradient=True,
       [ 3.14159265,  1.39460474, -0.44313783,  1.09757975, -1.09757975,
         0.44313783, -1.39460474,  3.14159265])


We can also verify the correctness of $\Phi$ from the corresponding circuit.

In [23]:
x = np.random.rand() * 2 - 1 # random x in [-1, 1]
cir = qsp.block_encoding(x)
print(cir)
print(f"the error of simulating P(x) is {np.abs(cir.unitary_matrix()[0, 0].item() - P(x))}")

--Rz(-6.28)----Rx(-1.74)----Rz(2.789)----Rx(-1.74)----Rz(-0.88)----Rx(-1.74)----Rz(2.195)----Rx(-1.74)----Rz(-2.19)----Rx(-1.74)----Rz(0.886)----Rx(-1.74)----Rz(-2.78)----Rx(-1.74)----Rz(-6.28)--
                                                                                                                                                                                                   
the error of simulating P(x) is 6.378721218542117e-08


### Variant of QSP

Moreover, the number of parameters can be further simplified to $k$ by finding a vector of angles $\Phi \in \mathbb{R}^k$ such that

$$
R_\Phi(x):=\prod_{i = 1}^{k} e^{i \phi_i \sigma_z} R(x)
= \begin{bmatrix}
    P(x) & ... \\
    ... & ...
\end{bmatrix}, \tag{6}
$$

where

$$
R(x) = \begin{bmatrix}
    x & \sqrt{1 - x^2} \\
    \sqrt{1 - x^2} & -x
\end{bmatrix}. \tag{7}
$$

In PaddleQuantum, we can compute such $\Phi$ by function `reflection_based_quantum_signal_processing`.

In [24]:
print(reflection_based_quantum_signal_processing(P))

[15.70796327 -0.17619159 -2.01393416 -0.47321657 -2.66837608 -1.12765849
 -2.96540107]


> Note: $R_{-\Phi}(x)$ is a block encoding of $P^*(x)$.

## Quantum Singular Value Transformation

To start with, we need to define the singular value transformation. 

Let $\mathcal{X} \in \mathbb{C}^{2^m \times 2^m}$ be a matrix. Suppose there exists a unitary $U$ and two orthogonal projectors $W$ and $V$ such that $\mathcal{X} = W U V$. Then there exists two orthonormal basis $\{|\omega_j\rangle\}_{j=1}^{2^m}$ and $\{|\nu_j\rangle\}_{j=1}^{2^m}$ extended by eigenstates of $W$ and $V$ respectively, such that

$$
\mathcal{X} = \sum_{j=1}^{2^m} \lambda_j |\omega_j\rangle \langle\nu_j|, \tag{8}
$$

where 

$$
\lambda_j = \begin{cases}
\langle\omega_j| U |\nu_j\rangle \text{  if } j \leq \text{rank}(\mathcal{X}) \\
0 \text{  otherwise }
\end{cases} \tag{9}
$$

is a singular value of $\mathcal{X}$. The **singular value transformation** (SVT) of $\mathcal{X}$ for a function $f$ is defined to be

$$
f^{(SV)}(\mathcal{X}) := \sum_{j=1}^{2^m} f(\lambda_j)
\begin{cases}
       |\nu_j\rangle\langle\nu_j| & \text{  if } f \text{ is even} \\
       |\omega_j\rangle\langle\nu_j| & \text{  if } f \text{ is odd}
\end{cases}. \tag{10}
$$

### Decomposition and QSP


Paper [[2]]((https://doi.org/10.1145/3313276.3316366)) proves that under **appropriate** choices of basis, $U, V$ and $W$ can be decomposed by subspaces of $\lambda_j$ such that

$$
\begin{split}
U & = \bigoplus_{j: \lambda_j \in [0, 1)} R(\lambda_j) \oplus
        \bigoplus_{j: \lambda_j = 1} [1] \oplus
        \bigoplus_{j: V|\nu_j\rangle = |\nu_j\rangle,\, WU|\nu_j\rangle = 0} [1] \oplus
        \bigoplus_{j: W|\omega_j\rangle = |\omega_j\rangle,\, VU^\dagger|\omega_j\rangle = 0} [1] \oplus 
        \bigoplus_{j: V|\nu_j\rangle = W|\omega_j\rangle = 0} [...], \\
V & = \bigoplus_{j: \lambda_j \in [0, 1)} |0\rangle\langle0| \oplus
        \bigoplus_{j: \lambda_j = 1} [1] \oplus
        \bigoplus_{j: V|\nu_j\rangle = |\nu_j\rangle,\, WU|\nu_j\rangle = 0} [1] \oplus
        \bigoplus_{j: W|\omega_j\rangle = |\omega_j\rangle,\, VU^\dagger|\omega_j\rangle = 0} [0]  \oplus 
        \bigoplus_{j: V|\nu_j\rangle = W|\omega_j\rangle = 0} [0] \text{ and}\\
W & = \bigoplus_{j: \lambda_j \in [0, 1)} |0\rangle\langle0| \oplus
        \bigoplus_{j: \lambda_j = 1} [1] \oplus
        \bigoplus_{j: V|\nu_j\rangle = |\nu_j\rangle,\, WU|\nu_j\rangle = 0} [0] \oplus
        \bigoplus_{j: W|\omega_j\rangle = |\omega_j\rangle,\, VU^\dagger|\omega_j\rangle = 0} [1]  \oplus 
        \bigoplus_{j: V|\nu_j\rangle = W|\omega_j\rangle = 0} [0] .\\
\end{split} \tag{11}
$$

We can decompose $f^{(SV)}$ in a similar manner.

$$
f^{(SV)}(\mathcal{X}) = \sum_{j: \lambda_j \in [0, 1)} f(\lambda_j)\,... + \sum_{j: \lambda_j = 1} f(1)\,... + \sum_{j: \lambda_j = 0, V |\nu_j\rangle \neq 0} f(0)\,... + \sum_{j: \lambda_j = 0, W |\omega_j\rangle \neq 0} f(0)\,... + \sum_{j: \lambda_j = 0, \text{otherwise}} f(0)\,...\,, \tag{12}
$$


Let's connect above decomposition with quantum signal processing. Suppose function $f$ is a polynomial $P \in \mathbb{C}[x]$ with degree $k$ that satisfies the requirements in the last section. 
Then there exists $\Phi \in \mathbb{R}^k$ such that $R_\Phi$ is a block encoding of $P$. Moreover, $P$ satisfies $P(1) = e^{i \sum_{j=1}^k \phi_j}$ and $P(0) = e^{i \sum_{j=1}^k (-1)^{j+1} \phi_j}$.
Now Define $U_\Phi$ as

$$
U_\Phi :=  \begin{cases}
& \prod_{j = 1}^{k / 2} e^{i\phi_{2j - 1} (2V - I)} U^\dagger e^{i\phi_{2j} (2W - I)} U \text{  if } k \text{ is even} \\
e^{i\phi_1 (2W - I)} U & \prod_{j = 1}^{(k - 1) / 2} e^{i\phi_{2j} (2V - I)} U^\dagger e^{i\phi_{2j+1} (2W - I)} U \text{  if } k \text{ is odd}
\end{cases}. \tag{13}
$$

Then

$$
U_\Phi =  \begin{cases}
        \bigoplus R_\Phi(\lambda_j) \oplus
        \bigoplus [e^{i \sum_{j=1}^k \phi_j}] \oplus
        \bigoplus [e^{i \sum_{j=1}^k (-1)^{j+1} \phi_j}] \oplus
        \bigoplus [e^{i \sum_{j=1}^k (-1)^{j+1} \phi_j}] \oplus 
        \bigoplus [...]
\text{  if } k \text{ is even} \\
        \bigoplus R_\Phi(\lambda_j) \oplus
        \bigoplus [e^{i \sum_{j=1}^k \phi_j}] \oplus
        \bigoplus [e^{i \sum_{j=1}^k (-1)^{j+1} \phi_j}] \oplus
        \bigoplus [e^{i \sum_{j=1}^k (-1)^{j+1} \phi_j}] \oplus 
        \bigoplus [...]
\text{  if } k \text{ is odd}
\end{cases} \tag{14}
$$

and hence

$$
P^{(SV)}(\mathcal{X}) = \begin{cases} 
        \bigoplus \begin{bmatrix}
                P(\lambda_j) & 0 \\
                0 & 0 \\
        \end{bmatrix} \oplus
        \bigoplus [P(1)] \oplus
        \bigoplus [P(0)] \oplus
        \bigoplus [0] \oplus 
        \bigoplus [0] = V U_\Phi V
\text{  if } k \text{ is even} \\ 
        \bigoplus \begin{bmatrix}
                P(\lambda_j) & 0 \\
                0 & 0 \\
        \end{bmatrix} \oplus
        \bigoplus [P(1)] \oplus
        \bigoplus [0] \oplus
        \bigoplus [0] \oplus 
        \bigoplus [0] = W U_\Phi V 
\text{  if } k \text{ is odd}
\end{cases}. \tag{15}
$$

If we can realize $V U_\Phi V$ or $W U_\Phi V$ by a quantum algorithm, then such transformation is the quantum singular value transformation of $\mathcal{X}$.

### QSVT and Block Encoding

Suppose $U$ is a block encoding unitary of $A$ with respect to orthogonal projectors $W$ and $V$. Then $\mathcal{X} = A \oplus 0I^{\otimes (m - n)}$ and $P^{(SV)}(\mathcal{X}) = P^{(SV)}(A) \oplus 0I^{\otimes (m - n)}$. Therefore, $U_\Phi$ is a block encoding of $P^{(SV)}(A)$.

When $W = V$, $\{|\omega_j\rangle \}_{j=1}^{2^m} = \{ |\nu_j\rangle \}_{j=1}^{2^m}$. Denote $P(x) := \sum_{i=0}^k c_i x^i$. We can find that 

$$
P^{(SV)}(\mathcal{X}) = \sum_{j=1}^{2^m} P(\lambda_j) |\nu_j\rangle \langle\nu_j| = P(X) = P(A) \oplus 0I^{\otimes (m - n)}. \tag{16}
$$ 

That is, when $W = V$, QSVT maps a block encoding of $A$ to a block encoding of $P(A)$.

PaddleQuantum has a built-in class ``QSVT`` for quantum singular value transformation when $W = V = |0\rangle^{\otimes (m - n)}\langle0|^{\otimes (m - n)} \otimes I^{\otimes n}$. We can call its member function ``QSVT.block_encoding_matrix()`` to verify the correctness of above theories, from entries to entries and from eigenvalues to eigenvalues.

In [25]:
qsvt = QSVT(P, U, num_block_qubits)

In [26]:
# find P(A) and its expected eigenvalues, note that they are computed in different ways
expect_PX = poly_matrix(P, A).numpy()
expect_eig_result = np.sort(list(map(lambda x: P(x), np.linalg.eigvals(A.numpy()))))

# Calculate U_\Phi and extract eigenvalues of block encoded matrix
U_Phi = qsvt.block_encoding_matrix().numpy()
actual_PX = U_Phi[0:2 ** num_block_qubits, 0:2 ** num_block_qubits]
actual_eig_result = np.sort(np.linalg.eigvals(actual_PX))

In [27]:
print("error of simulating P(X)")
print("     maximum absolute, ", np.amax(abs(expect_PX - actual_PX)))
print("     percentage, ", np.linalg.norm(expect_PX - actual_PX) / np.linalg.norm(expect_PX))

error of simulating P(X)
     maximum absolute,  1.6997650495437753e-07
     percentage,  3.4201093195057237e-07


In [28]:
print("error of eigenvalues of simulating P(X)")
print("     maximum absolute, ", np.amax(abs(expect_eig_result - actual_eig_result)))
print("     percentage, ", np.linalg.norm(expect_eig_result - actual_eig_result) / np.linalg.norm(expect_eig_result))

error of eigenvalues of simulating P(X)
     maximum absolute,  2.3308557146996258e-07
     percentage,  2.2962108806419593e-07


### Qubitization: Quantum Realization of $U_\Phi$

One question is, how to realize $U_\Phi$ by a quantum circuit? Note that we assume $U$ is implementable. Then the remaining problem is the realization of unitaries $e^{i\phi (2W - I)}$ and $e^{i\phi (2V - I)}$, which are essentially $R_Z(-2\phi)$ operators performed in projection spaces of $W$ and $V$ respectively.

To achieve this, Lemma 10 in paper [[1]](https://quantum-journal.org/papers/q-2019-07-12-163/) projects the image space of projector into an ancilla qubit and perform the rotation operation on that qubit. Then by entanglement such rotation is simultaneously applied to the main register. 

The results are summarized to the following circuit:

![U_Phi](figures/QSVT-fig-U_Phi.png "Figure 2: Quantum Circuit for QSVT, where k is the degree of P")

When $W = V = |0\rangle^{\otimes (m - n)}\langle0|^{\otimes (m - n)} \otimes I^{\otimes n}$, PaddleQuantum can create a circuit in Figure 2 by member function ``QSVT.block_encoding_circuit``. We can show that the constructed quantum circuit can simulate $U_\Phi$, by comparison of the output state and $(U_\Phi \otimes I)|\psi\rangle|0\rangle$ for $|\psi\rangle \in \mathbb{C}^{2^m}$.

In [29]:
# set state vector backend
paddle_quantum.set_backend('state_vector')

# arbitrary state such that last qubit is in state |0\rangle
ket_0 = [1.0, 0.0]
psi = np.array([np.random.rand() + 1j * np.random.rand() for _ in range(2 ** num_qubits)])
psi = np.kron(psi / np.linalg.norm(psi), ket_0)

In [30]:
expect_state = np.kron(U_Phi, np.eye(2)) @ psi

cir = qsvt.block_encoding_circuit()
actual_state = cir(paddle_quantum.State(psi, dtype='complex64')).data.numpy()

print(f"the different between expected and actual state is {np.linalg.norm(expect_state - actual_state)}")

the different between expected and actual state is 2.383485634049416e-07


> Note: if we add Hadamard gates at the beginning and the end of the ancilla register in Figure 2, then the quantum circuit turns to simulate the real part of the polynomial. Combined with the technique of linear combination of unitaries, theocratically we can simulate all complex polynomials with norm smaller than 1 using quantum circuit.

## Application: Amplitude Amplification

Let $|\psi\rangle$ be a $m$-qubit quantum state such that $|\psi\rangle = \sin(\theta)|\psi_{\text{good}}\rangle + \cos(\theta) |\psi_{\text{bad}}\rangle$. **Amplitude amplification** is a quantum algorithm that can increase the amplitude of $|\psi_{\text{good}}\rangle$ to approximately 1.
This algorithm can be viewed in a different prospective. Let $U$ be the unitary and $W, V$ be two orthogonal projectors such that $|\psi\rangle = U V |0\rangle^{\otimes m}$ and $\sin(\theta)|\psi_{\text{good}}\rangle = W |\psi\rangle$, implying $\sin(\theta)|\psi_{\text{good}}\rangle = W U V |0\rangle^{\otimes m}$. Therefore, amplitude amplification is essentially a singular value transformation of $\mathcal{X} = W U V$. In this section we will show how to use QSVT to do fixed-point (i.e. $\theta = \frac{\pi}{2k}$ for some $k \in \mathbb{Z}$) amplitude amplification from Theorem 28 of paper [[2]]((https://doi.org/10.1145/3313276.3316366)). 

Suppose $\sin(\theta)|\psi_{\text{bad}}\rangle = (|\psi_{\text{good}}\rangle^{\otimes (m - n)}\langle0|^{\otimes (m - n)} \otimes I^{\otimes n}) |\psi\rangle$ and $\theta = \frac{\pi}{2k}$ for some $k \in \mathbb{Z}$. Then $W = V = |0\rangle^{\otimes (m - n)}\langle0|^{\otimes (m - n)} \otimes I^{\otimes n}$ and $\mathcal{X} = A \oplus 0I^{\otimes (m - n)}$, where $A$ is the left-upper block of $U$. 

Observe that $\mathcal{X} |\psi_{\text{good}\rangle^{\otimes m} = \sin(\frac{\pi}{2k}) |\psi\rangle}$. Therefore, we aim to find a quantum circuit with unitary $U_\Phi$ such that $W U_\Phi V = \frac{1}{\sin(\frac{\pi}{2k})} \mathcal{X}$, implying $W U_\Phi V |\psi_{\text{good}}\rangle^{\otimes m} = |0\rangle$. By $\mathcal{X} = W U V$, the absolute values of singular values of $\mathcal{X}$ are $\sin(\frac{\pi}{2k})$. Moreover, choose $P(x) = (-1)^k T_k (x)$, where $T_k$ is the Chebyshev polynomial of the first kind of order $k$. Then

$$
P(\sin(\frac{\pi}{2k})) = (-1)^k T_k (\sin(\frac{\pi}{2k})) = (-1)^k \cos(\frac{k - 1}{2}\pi) = 1, \tag{17}
$$

implies that the QSVT of $\mathcal{X}$ in terms of polynomial $P$ is a block encoding of $B := \frac{1}{\sin(\frac{\pi}{2k})} A$, as required.

We set $k = 3$ so that $\sin(\frac{\pi}{2k}) = \frac{1}{2}$, and $U$ is a randomly chosen unitary. We want to show that the left-upper block of $U$ is amplified by 2 after QSVT.

In [31]:
P = -1 * Chebyshev([0 for _ in range(3)] + [1]).convert(kind=Polynomial)
U = unitary_random_with_hermitian_block(num_qubits, is_unitary=True)
A = U[0:2 ** num_block_qubits, 0:2 ** num_block_qubits].numpy()

amplifier = QSVT(P, U, num_block_qubits)
U_Phi = amplifier.block_encoding_unitary()

In [32]:
B = U_Phi[0:2 ** num_block_qubits, 0:2 ** num_block_qubits].numpy()
print(f"the accuracy of quantum singular value transformation is {np.linalg.norm(B - 2 * A)}")

the accuracy of quantum singular value transformation is 3.029211939065135e-07


As shown above, we successfully amplify $\mathcal{X}$ by 2.

---
## References

[1] Low, Guang Hao, and Isaac L. Chuang. "Hamiltonian simulation by qubitization." [Quantum 3 (2019): 163.](https://doi.org/10.22331/q-2019-07-12-163)

[2] Gilyén, András, et al. "Quantum singular value transformation and beyond: exponential improvements for quantum matrix arithmetics." [Proceedings of the 51st Annual ACM SIGACT Symposium on Theory of Computing. 2019.](https://doi.org/10.1145/3313276.3316366)

[3] Camps, Daan, et al. "Explicit Quantum Circuits for Block Encodings of Certain Sparse Matrices." [arXiv preprint arXiv:2203.10236 (2022).](http://arxiv.org/abs/2203.10236)

[4] Clader, B. David, et al. "Quantum Resources Required to Block-Encode a Matrix of Classical Data." [arXiv preprint arXiv:2206.03505 (2022).](http://arxiv.org/abs/2206.03505)
